In [ ]:
import sys, os

sys.path.insert(0, os.path.abspath("../../"))

In [ ]:
from itertools import product
from typing import TYPE_CHECKING, Optional
from collections import OrderedDict

from numpy import array, sign, sqrt, cos, pi, sin
from numpy.random import rand


from pydistsim.conf import settings
from pydistsim.logger import logger
from pydistsim.network.environment import Environment2D
from pydistsim.network.node import Node
from pydistsim.network.rangenetwork import BidirectionalRangeNetwork, RangeNetwork
from pydistsim.network.network import BidirectionalNetwork, Network, NetworkType

In [ ]:
def get_ring_pos(n: int, env: "Environment2D"):
    env_shape = env.image.shape
    center = (env_shape[0] // 2, env_shape[1] // 2)
    radius = 0.82 * (min(env_shape) / 2)  # 82% of the smallest dimension, leaves enough space for the node labels.

    nodes = []
    for i in range(n):
        angle = 2 * i * pi / n
        x = center[0] + radius * cos(angle)
        y = center[1] + radius * sin(angle)
        nodes.append((x, y))
    return nodes


def get_complete_network(n: int, network_type: "NetworkType" = BidirectionalNetwork):
    net = network_type()
    node_pos_list = get_ring_pos(n, net.environment)
    nodes = [net.add_node(pos=node_pos_list) for node_pos_list in node_pos_list]

    for i, j in product(range(n), range(n)):
        if i != j:
            net.add_edge(nodes[i], nodes[j])

    return net


def get_ring_network(n: int, network_type: "NetworkType" = BidirectionalNetwork):
    net = network_type()
    node_pos_list = get_ring_pos(n, net.environment)
    nodes = [net.add_node(pos=node_pos_list) for node_pos_list in node_pos_list]

    for i in range(n):
        net.add_edge(nodes[i], nodes[(i + 1) % n])

    return net


get_complete_network(5).show()

In [ ]:
def get_hypercube_network(dimension: int, network_type: "NetworkType" = BidirectionalNetwork):
    LABEL_KEY = "HYPERCUBE_LABEL"

    def create_hypercube(dim):
        # Recursive function to create the hypercube
        # A hypercube of dimension n is the disjoint union of two hypercubes of dimension n-1, forming a perfect
        # matching between the nodes of the two hypercubes.
        node_pos = OrderedDict()
        edges = []
        dist = 1

        if dim == 0:
            node = Node()
            node.memory[LABEL_KEY] = "0"
            node_pos[node] = (0, 0)
        elif dim == 1:
            node1 = Node()
            node1.memory[LABEL_KEY] = "0"
            node2 = Node()
            node2.memory[LABEL_KEY] = "1"
            node_pos[node1] = (0, 0)
            node_pos[node2] = (dist, 0)
            edges.append((node1, node2))
        else:
            part_1_nodes, part_1_edges = create_hypercube(dim - 1)
            part_2_nodes, part_2_edges = create_hypercube(dim - 1)

            edges += part_1_edges
            edges += part_2_edges

            part_1_width = max([pos[0] for pos in part_1_nodes.values()])
            part_1_height = max([pos[1] for pos in part_1_nodes.values()])

            if dim == 1:  # Grow only in the x-axis
                x_offset = dist
                y_offset = 0
            elif dim == 2:  # Grow only in the y-axis
                x_offset = 0
                y_offset = dist
            elif dim == 3:  # Grow equally in both axes
                x_offset = dist / 2
                y_offset = dist / 2
            elif dim % 2 == 0:  # Even dimensions grow in the x-axis
                x_offset = part_1_width * 1.1
                y_offset = dist / 4
            else:  # Odd dimensions grow in the y-axis
                x_offset = dist / 4
                y_offset = part_1_height * 1.1

            for node, pos in part_1_nodes.items():
                node_pos[node] = pos
                node.memory[LABEL_KEY] = "0" + node.memory[LABEL_KEY]
            for node, pos in part_2_nodes.items():
                node_pos[node] = (pos[0] + x_offset, pos[1] + y_offset)
                node.memory[LABEL_KEY] = "1" + node.memory[LABEL_KEY]

            for node1, node2 in zip(part_1_nodes.keys(), part_2_nodes.keys()):
                edges.append((node1, node2))

        return node_pos, edges

    # Create the nodes
    node_pos_list, edges = create_hypercube(dimension)

    # Create the network
    net = network_type()
    canvas_shape = net.environment.image.shape
    usable_shape = (canvas_shape[1] * 0.82, canvas_shape[0] * 0.82)

    # Normalize the positions
    max_x = max(max([pos[0] for pos in node_pos_list.values()]), 1)
    max_y = max(max([pos[1] for pos in node_pos_list.values()]), 1)

    for node, pos in node_pos_list.items():
        # If the dimension is 0 or 1, the nodes are in the center
        x = pos[0] if dimension != 0 else 0.5
        y = pos[1] if dimension not in (0, 1) else 0.5
        final_pos = (x / max_x * usable_shape[0], y / max_y * usable_shape[1])

        # Add the node
        net.add_node(node, pos=final_pos)
        net.labels[node] = node.memory[LABEL_KEY]

        # Remove the label from the memory as it represents some knowledge of the network
        del node.memory[LABEL_KEY]

    # Add the edges to the network
    for node1, node2 in edges:
        net.add_edge(node1, node2)

    return net


get_hypercube_network(4).show()

In [ ]:
from pydistsim.network.generator import NetworkGenerator

In [ ]:
NetworkGenerator(15, enforce_connected=True).generate_homogeneous_network().show()